In [ ]:
import os
os.environ["SM_FRAMEWORK"] = "tf.keras"
import glob
from tensorflow.keras.models import load_model
import segmentation_models as sm
import tensorflow.keras as keras
import tensorflow as tf
import rasterio
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.io

In [ ]:
def create_if_not_exist(directory):
    if not os.path.exists(directory):
        os.mkdir(directory)

In [ ]:
model_name = 'FPN'
model_num = "15normal"

parent_folder = 'path/to/parent/folder'

csv_path = os.path.join(parent_folder, 'results', model_name, str(model_num), 'train.csv')
model_folder = os.path.join(parent_folder, 'results', model_name, str(model_num), 'model')
figs_folder = os.path.join(parent_folder, 'results', model_name, str(model_num), 'figs')

create_if_not_exist(figs_folder)

model_name = os.listdir(model_folder)[-1]
model_path = os.path.join(model_folder, model_name)

print(model_num, csv_path, model_path, sep='\n')

In [ ]:
model = load_model(model_path)


In [ ]:
a = scipy.io.loadmat(r'path/to/a/sample/horse_1.mat')
a.keys()
normals = a['Normals_gt']
images = a['images']
mask = a['mask']

normals.shape, images.shape, mask.shape

In [ ]:

new_predicted_image = np.zeros_like(normals)
count_predicted_image = np.zeros(normals.shape[:-1])
stride = 32
patch_size = 128



i_range = range(0, 1024-patch_size, stride)
j_range = range(0, 1224-patch_size, stride)

counter = 0
total = len(j_range) * len(i_range)

for i in i_range:
    for j in j_range:
        counter += 1
        if counter % 10 == 0:
            print("{} / {}".format(counter, total), end='\r')
        current_patch = images[np.newaxis, i:i+patch_size, j:j+patch_size]
        predict_current = model.predict(current_patch)[0, ..., :]
        new_predicted_image[i:i+patch_size, j:j+patch_size, :] += predict_current
        count_predicted_image[i:i+patch_size, j:j+patch_size] += 1



In [ ]:
plt.imshow(count_predicted_image)
plt.colorbar()

In [ ]:
plt.imshow(new_predicted_image)

In [ ]:
normals_hat = new_predicted_image.copy() / count_predicted_image[..., np.newaxis]

normals_hat = normals_hat / np.linalg.norm(normals_hat, axis=2, keepdims=True)
normals_hat[mask==0] = 0

fig, axes = plt.subplots(2, 2, figsize=(20,20))
axes = axes.ravel()

axes[3].imshow(normals_hat)
axes[1].imshow(normals)
axes[2].imshow(normals - normals_hat)
axes[0].imshow(images[..., 0])

In [ ]:
a = normals[mask == 1] - normals_hat[mask == 1]
aa = plt.hist(a.ravel(), bins=20)

In [ ]:
normals.min(), normals.max(), normals_hat.min(), normals_hat.max(), 

In [ ]:
plt.imshow(np.linalg.norm(normals, axis=2))
plt.colorbar()

In [ ]:
def calculate_mae(pred: np.ndarray, label: np.ndarray):
    adotb = pred * label
    norma = np.linalg.norm(pred, axis=1, keepdims=True)
    normb = np.linalg.norm(label, axis=1, keepdims=True)
    ae = np.arccos(np.sum(adotb, axis=1, keepdims=True) / norma / normb)
    return np.mean(ae)

mae = calculate_mae(normals_hat[mask == 1], normals[mask == 1])
np.rad2deg(mae)